# Vanilla LSTM Probe
**Goal**: Test LSTM as architectural diversity source for ensemble.
Same recipe as winning GRU (h=64, 3L, raw32, no norm, MSE, dropout=0).
LSTM has separate cell state (long-term memory) vs GRU's simpler gates.
Different inductive bias should produce decorrelated predictions.

**Plan**: 5 screening seeds. If mean val within 0.005 of GRU base (0.2689), include in greedy pool.

In [ ]:
# Cell 0: Mount Drive, download data from Kaggle
import os, json

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "FILL_IN"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

In [ ]:
# Cell 1: Setup -- clone repo, link data
import os, subprocess
REPO = "/content/competition_package"

os.chdir("/content")
subprocess.run(["rm", "-rf", REPO], check=False)
subprocess.run(["git", "clone", "https://github.com/vincentvdo6/competition_package.git", REPO], check=True)
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

subprocess.run(["ln", "-sf", "/content/data/train.parquet", "datasets/train.parquet"], check=True)
subprocess.run(["ln", "-sf", "/content/data/valid.parquet", "datasets/valid.parquet"], check=True)

assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"
commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip()
print(f"Commit: {commit}")
print(f"GPU: {subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], text=True).strip()}")
print("Ready!")

In [ ]:
# Cell 2: Train vanilla LSTM x 5 seeds
import os, subprocess, sys
os.chdir("/content/competition_package")

CONFIG = "configs/vanilla_lstm_v1.yaml"
SEEDS = [42, 43, 44, 45, 46]

print(f"=== VANILLA LSTM PROBE ===")
print(f"Config: {CONFIG}")
print(f"Seeds: {SEEDS}")
print("=" * 60, flush=True)

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"Training vanilla_lstm_v1 seed {seed}")
    print(f"{'='*60}", flush=True)
    proc = subprocess.Popen(
        [sys.executable, "-u", "scripts/train.py",
         "--config", CONFIG,
         "--seed", str(seed), "--device", "cuda"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    proc.wait()
    if proc.returncode != 0:
        print(f"ERROR: seed {seed} failed with rc={proc.returncode}")

print(f"\nAll {len(SEEDS)} LSTM seeds done!")

In [ ]:
# Cell 3: Evaluate LSTM vs GRU base
import os, glob, torch
os.chdir("/content/competition_package")

GRU_BASE_MEAN = 0.2689  # mean of 23 GRU base seeds

results = []
for pt in sorted(glob.glob("logs/vanilla_lstm_v1_seed*.pt")):
    basename = os.path.basename(pt)
    if '_epoch' in basename:
        continue
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    score = float(ckpt.get("best_score", 0))
    epoch = ckpt.get("best_epoch", "N/A")
    seed = int(basename.split("seed")[1].split(".")[0])
    results.append((seed, basename, score, epoch))

results.sort(key=lambda x: -x[2])

print(f"{'Rank':<6} {'Seed':<6} {'Val Score':>10} {'Epoch':>6}")
print("-" * 35)
for i, (seed, name, score, epoch) in enumerate(results, 1):
    print(f"{i:<6} s{seed:<5} {score:>10.4f} {str(epoch):>6}")

if results:
    scores = [s for _, _, s, _ in results]
    mean_val = sum(scores) / len(scores)
    delta = mean_val - GRU_BASE_MEAN
    print(f"\nLSTM: {len(results)} seeds, mean={mean_val:.4f}, delta={delta:+.4f} vs GRU base")
    print(f"Best: {max(scores):.4f}, Worst: {min(scores):.4f}")
    if delta >= -0.005:
        print(">>> PASS: Include in greedy ensemble pool <<<")
    else:
        print(">>> FAIL: LSTM too weak, KILL <<<")

In [ ]:
# Cell 4: Strip checkpoints + zip + save to Drive
import os, torch, glob, shutil
os.chdir("/content/competition_package")
os.makedirs("logs/slim", exist_ok=True)

for pt in sorted(glob.glob("logs/vanilla_lstm_v1_seed*.pt")):
    basename = os.path.basename(pt)
    if '_epoch' in basename:
        continue
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    slim = {
        "model_state_dict": ckpt["model_state_dict"],
        "config": ckpt.get("config", {}),
        "best_score": ckpt.get("best_score", None),
        "best_epoch": ckpt.get("best_epoch", None),
    }
    out = f"logs/slim/{basename}"
    torch.save(slim, out)
    orig = os.path.getsize(pt) / 1e6
    new = os.path.getsize(out) / 1e6
    print(f"{basename}: {orig:.1f}MB -> {new:.1f}MB")

shutil.make_archive("/content/lstm_probe_seeds", "zip",
                     "/content/competition_package/logs/slim")
sz = os.path.getsize("/content/lstm_probe_seeds.zip") / 1e6
print(f"\nlstm_probe_seeds.zip: {sz:.1f}MB")

shutil.copy("/content/lstm_probe_seeds.zip",
            "/content/drive/MyDrive/wunderfund/lstm_probe_seeds.zip")
print("Saved to Drive: MyDrive/wunderfund/lstm_probe_seeds.zip")